In [1]:
import os
import sys
import subprocess
import glob
import pathlib

# logic to accomodate Google Colab
try:
    import google.colab

    ENV_IS_CL = True

    root = pathlib.Path("/content/t-route").resolve()
    subprocess.run(
        [
            "git",
            "clone",
            "https://github.com/NOAA-OWP/t-route.git",
        ]
    )

    ! pip install geopandas
    ! pip install netcdf4
    ! pip install gif

except:
    ENV_IS_CL = False
    root = pathlib.Path("..").resolve()

fortran_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "MC_singleSeg_singleTS")
sys.path.append(fortran_source_dir)

routing_v02_dir = os.path.join(root, "src", "python_routing_v02","fast_reach")
sys.path.append(routing_v02_dir)

reservoir_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "Reservoir_singleTS")
sys.path.append(reservoir_source_dir)

sys.path.append(os.path.join(root, "src", "python_framework_v02"))
sys.path.append(os.path.join(root, "src", "python_framework_v01"))

# import scientific libraries
import time
import numpy as np
from operator import itemgetter
from itertools import chain, islice
from functools import partial
from joblib import delayed, Parallel
import random
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

# network/reach utilities & routing module
import nhd_network_utilities_v02 as nnu
import nhd_io
import nhd_network
import network_dl

os.chdir(routing_v02_dir)
! ./cython_compile.sh
os.chdir(os.path.join(root,"notebooks"))

import mc_reach

rm: cannot remove '*.so': No such file or directory
Compiling /mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/python_routing_v02/fast_reach/fortran_wrappers.pxd
Compiling /mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/python_routing_v02/fast_reach/mc_reach.pyx
Compiling /mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/python_routing_v02/fast_reach/reach.pyx
Compiling /mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/python_routing_v02/fast_reach/reservoir.pyx
Compiling /mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/python_routing_v02/fast_reach/utils.pyx
In file included from /home/awlostowski/tr/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832,
                 from /home/awlostowski/tr/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/awlostowski/tr/lib/python3.8/site-packages/numpy/core/include/numpy/arrayo

In [2]:
# number of simulated timesteps
nts = 100

# duration of each timestep (seconds)
dt = 300

print("The total simulated time is %s days" % (nts * dt / 60 / 60 / 24))


The total simulated time is 0.34722222222222227 days


In [3]:
print("Identifying supernetwork connections set")

test_folder = os.path.join(root, r"test")
geo_input_folder = os.path.join(test_folder, r"input", r"geo")
supernetwork = "Mainstems_CONUS"

network_data = nnu.set_supernetwork_data(
    supernetwork=supernetwork, geo_input_folder=geo_input_folder
)

# if the NHDPlus RouteLink file does not exist, download it.
if not os.path.exists(network_data["geo_file_path"]):
    filename = os.path.basename(network_data["geo_file_path"])
    network_dl.download(network_data["geo_file_path"], network_data["data_link"])

# select only the necessary columns of geospatial data, set the DataFrame index
cols = [v for c, v in network_data["columns"].items()]
data = nhd_io.read(network_data["geo_file_path"])
data = data[cols]
data = data.set_index(network_data["columns"]["key"])

# mask NHDNetwork to isolate test network of choice
if "mask_file_path" in network_data:
    data_mask = nhd_io.read_mask(
        network_data["mask_file_path"], layer_string=network_data["mask_layer_string"],
    )
    data = data.filter(data_mask.iloc[:, network_data["mask_key"]], axis=0)

# sort index
data = data.sort_index()

# replace downstreams
data = nhd_io.replace_downstreams(data, network_data["columns"]["downstream"], 0)

# extract downstream connections for each node
connections = nhd_network.extract_connections(data, network_data["columns"]["downstream"])

print("supernetwork connections set created.")

print("Organizing segments into reaches.")

# reverse the network - track upstream connections
rconn = nhd_network.reverse_network(connections)

# isolate independent subnetworks
subnets = nhd_network.reachable_network(rconn)

# identify the segments in each subnetwork
subreachable = nhd_network.reachable(rconn)

# break each subnetwork into reaches
subreaches = {}
for tw, net in subnets.items():
    path_func = partial(nhd_network.split_at_junction, net)
    subreaches[tw] = nhd_network.dfs_decomposition(net, path_func)

print("Reach creation complete.")

def step_qlats(data, nsteps, qlat):

    q1 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q2 = np.full((len(data.index), nsteps // 10), qlat, dtype="float32")
    q3 = np.full((len(data.index), nsteps // 10), qlat, dtype="float32")
    q4 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q5 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q6 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q7 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q8 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q9 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")
    q10 = np.full((len(data.index), nsteps // 10), 0, dtype="float32")

    q = np.concatenate((q1, q2, q3, q4, q5, q6, q7, q8, q9, q10), axis=1)

    ql = pd.DataFrame(q, index=data.index, columns=range(nsteps))

    return ql


# create the lateral inflow data
qlats = step_qlats(data, nts, 10.0)

# add a dt column to the data DataFrame
data["dt"] = dt
# rename columns to specific variable names expected by mc_reach.compute_network
column_rename = {
    network_data["columns"]["dx"]: "dx",
    network_data["columns"]["tw"]: "tw",
    network_data["columns"]["twcc"]: "twcc",
    network_data["columns"]["bw"]: "bw",
    network_data["columns"]["ncc"]: "ncc",
    network_data["columns"]["s0"]: "s0",
    network_data["columns"]["cs"]: "cs",
    network_data["columns"]["n"]: "n",
}

data = data.rename(columns=column_rename)
# change variables to type float32, as expected by mc_reach.compute_network
data = data.astype("float32")
# assume zero flow and depth initial conditions
q0 = pd.DataFrame(0,index = data.index, columns = ["qu0","qd0","h0"], dtype = "float32")

Identifying supernetwork connections set
supernetwork connections set created.
Organizing segments into reaches.
Reach creation complete.


# serial simulation

In [4]:

start_time = time.time()
results = []
for twi, (tw, reach) in enumerate(subreaches.items(), 1):
    r = list(chain.from_iterable(reach))
    data_sub = data.loc[
        r, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
    ].sort_index()
    qlat_sub = qlats.loc[r].sort_index()
    q0_sub = q0.loc[r].sort_index()
    results.append(
        mc_reach.compute_network(
            nsteps = nts,
            reaches = reach,
            connections = subnets[tw],
            data_idx = data_sub.index.values,
            data_cols = data_sub.columns.values,
            data_values = data_sub.values.astype("float32"),
            qlat_values = qlat_sub.values.astype("float32"),
            initial_conditions = q0_sub.values.astype("float32"), # !! pass initial conditions, here !!
            assume_short_ts = True
        )
    )
end_time = time.time()

# create a multi-index DataFrame with flow, depth, and velocity simulations
fdv_columns = pd.MultiIndex.from_product([range(nts), ["q", "v", "d"]])
flowveldepth = pd.concat(
    [pd.DataFrame(d, index=i, columns=fdv_columns) for i, d in results], copy=False
)
flowveldepth = flowveldepth.sort_index()

print("timing:",end_time - start_time)


timing: 23.20840573310852


# multithreaded simulation 

In [ ]:
tuple_reaches = {}
for tw, net in subnets.items():
    path_func = partial(nhd_network.split_at_junction, net)
    tuple_reaches[tw] = nhd_network.dfs_decomposition_depth_tuple(net, path_func)

# convert tuple_reaches (dict) to a (list)
overall_tuple_reaches = []
for keys, tuple_list in tuple_reaches.items():
        overall_tuple_reaches.extend(tuple_list)

# create a dict with key = reach order, and values are segments in each reach of the corresponding order
overall_ordered_reaches_dict = nhd_network.tuple_with_orders_into_dict(overall_tuple_reaches)
max_order = max(overall_ordered_reaches_dict.keys())

overall_ordered_reaches_list = []
ordered_reach_count = []
ordered_reach_cache_count = []
for o in range(max_order,-1,-1):
    overall_ordered_reaches_list.extend(overall_ordered_reaches_dict[o])
    ordered_reach_count.append(len(overall_ordered_reaches_dict[o]))
    ordered_reach_cache_count.append(sum(len(r) for r in overall_ordered_reaches_dict[o]))
    
rconn_ordered = {}
rconn_ordered_byreach = {}
for o in range(max(overall_ordered_reaches_dict.keys()),-1,-1):
    rconn_ordered[o] = {}
    for reach in overall_ordered_reaches_dict[o]:
        for segment in reach:
            rconn_ordered[o][segment] = rconn[segment]
            rconn_ordered_byreach[segment] = rconn[segment]
                       
# prep parameter, lateral inflow, and initial condition data to be fed to routing model
r = list(chain.from_iterable(overall_ordered_reaches_list))
data_sub = data.loc[
    r, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
].sort_index()
# prep lateral inflow data
qlat_sub = qlats.loc[r].sort_index()
# prep initial conditions data
q0_sub = q0.loc[r].sort_index()

results = []

start_time = time.time()
results.append(
    mc_reach.compute_network_multithread(
        nts,
        overall_ordered_reaches_list,
        rconn_ordered_byreach,
        data_idx = data_sub.index.values,
        data_cols = data_sub.columns.values,
        data_values = data_sub.values.astype("float32"),
        qlat_values = qlat_sub.values.astype("float32"),
        initial_conditions = q0_sub.values.astype("float32"), # !! pass initial conditions, here !!
        reach_groups = np.array(ordered_reach_count, dtype="int32"),
        reach_group_cache_sizes = np.array(ordered_reach_cache_count, dtype="int32"),
        assume_short_ts = True
    )
)
end_time = time.time()

# create a multi-index DataFrame with flow, depth, and velocity simulations
fdv_columns = pd.MultiIndex.from_product([range(nts), ["q", "v", "d"]])
flowveldepth_mt = pd.concat(
    [pd.DataFrame(d, index=i, columns=fdv_columns) for i, d in results], copy=False
)
flowveldepth_mt = flowveldepth_mt.sort_index()

print("timing:",end_time - start_time)

In [ ]:
if flowveldepth_mt.shape[0] == flowveldepth.shape[0]:
    print("simulation domains are the same")